In [1]:
# ---- Imports ----
import pennylane as qml
from pennylane import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib

In [2]:
# ---- Data Preparation ----
df = pd.read_csv("UBER.csv")
scaler = MinMaxScaler()
df["Close_scaled"] = scaler.fit_transform(df[["Close"]])
joblib.dump(scaler, "uber_scaler.pkl")

['uber_scaler.pkl']

In [3]:
n_steps = 10
X, y = [], []
for i in range(len(df) - n_steps):
    X.append(df["Close_scaled"].values[i:i+n_steps])
    y.append(df["Close_scaled"].values[i+n_steps])
X, y = np.array(X), np.array(y)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [5]:
# ---- Quantum Circuit ----
n_qubits = 4  # we’ll use only 4 time steps as features
n_layers = 3
dev = qml.device("default.qubit", wires=n_qubits)

In [6]:

@qml.qnode(dev, interface="torch")
def circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

In [7]:
# ---- Torch Model ----
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.TorchLayer(circuit, weight_shapes)

In [8]:
# qlayer outputs shape: [batch_size, 1]
model = nn.Sequential(qlayer)  # no need for Linear(1,1) now
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

In [9]:
# ---- Convert Data ----
X_train_t = torch.tensor(X_train[:, :n_qubits], dtype=torch.float32)  # trim to 4 features
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

In [10]:
# ---- Train Loop ----
epochs = 20
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_train_t)
    loss = loss_fn(output, y_train_t)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

c:\Users\Madeehah\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([572, 1])) that is different to the input size (torch.Size([572])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 5/20, Loss: 0.205801
Epoch 10/20, Loss: 0.162371
Epoch 15/20, Loss: 0.123867
Epoch 20/20, Loss: 0.094370


In [11]:
torch.save(model.state_dict(), "uber_qnn_model.pt")
print("✅ Quantum model weights saved as uber_qnn_model.pt")


✅ Quantum model weights saved as uber_qnn_model.pt
